In [1]:
!pip install --quiet faiss-cpu datasets
!pip install --quiet  -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.6 MB/s eta 0:00:00


In [2]:
!pip install laserembeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 818.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 56.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883964 sha256=00716dd30047721ce4e2125ffa4ec9766bbd4905248d489db990da0c2f6e4d75
  Stored in directory: /root/.cache/pip/wheels/8e/12/4b/5c9eeed3636a4041c004e859e03429a49105672c7fb09ba6d9
Successfully built sacremoses
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uni

In [59]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [4]:
!huggingface-cli login --token <your_token_here>

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [36]:
from collections import Counter

def compute_metrics(ground_truth, predictions):
    unique_labels = set(ground_truth + predictions)
    true_positives = Counter()
    false_positives = Counter()
    false_negatives = Counter()

    for true_label, predicted_label in zip(ground_truth, predictions):
        if true_label == predicted_label:
            true_positives[true_label] += 1
        else:
            false_positives[predicted_label] += 1
            false_negatives[true_label] += 1

    precision = {}
    recall = {}
    f1_score = {}

    for label in unique_labels:
        precision[label] = true_positives[label] / (true_positives[label] + false_positives[label]) if (true_positives[label] + false_positives[label]) else 0
        recall[label] = true_positives[label] / (true_positives[label] + false_negatives[label]) if (true_positives[label] + false_negatives[label]) else 0
        f1_score[label] = 2 * (precision[label] * recall[label]) / (precision[label] + recall[label]) if (precision[label] + recall[label]) else 0

    macro_precision = sum(precision.values()) / len(unique_labels)
    macro_recall = sum(recall.values()) / len(unique_labels)
    macro_f1_score = sum(f1_score.values()) / len(unique_labels)
    accuracy = sum(true_positives.values()) / len(ground_truth)

    return accuracy, macro_precision, macro_recall, macro_f1_score

# Example usage:
ground_truth = ['a', 'b', 'c', 'a', 'b', 'c']
predictions = ['a', 'b', 'c', 'c', 'b', 'a']
accuracy, precision, recall, f1_score = compute_metrics(ground_truth, predictions)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}")


Accuracy: 0.6666666666666666
Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1 Score: 0.6666666666666666


In [5]:
import pandas as pd
df = pd.read_excel('train set 7 february 2024 from V6 reshaped.xlsx')
df = df[['text','label','language']]
df

,text,label,language
0,"i want a refund, help me egt one",get_refund,en
1,"I want to download an invoice, can you show me...",get_invoice,en
2,can u ask an agent how I can obtain an invoice?,get_invoice,en
3,How do I dispute a charge on my credit card?,idoos,en
4,At what age can a person open an account?,age_limit,en
...,...,...,...
5775,كيفاش نقدر نلغي تحويلة دارت عبر حوالة مالية؟,cancel_transfer,ar
5776,ما بغيتش حسابي، عاونوني نمسحوه,delete_account,ar
5777,تم الغاء الحفلة، غادي تحصّل ردّة.,get_refund,ar
5778,ما كتعاونيش، أشنو لازم ندير باش نهدر مع إنسان؟,contact_human_agent,ar


In [20]:
import pandas as pd
dtest_darija = pd.read_excel('test_set_darija_8fev24.xlsx')
dtest_arabe = pd.read_excel('test_set_arabe_8fev24.xlsx')

In [49]:
from sentence_transformers import SentenceTransformer

pretrained_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
"""you can use the same code with the other models namely
intfloat/multilingual-e5-base
sentence-transformers/LaBSE
sentence-transformers/paraphrase-multilingual-mpnet-base-v2
sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
"""
embeddings_normed = pretrained_model.encode(df['text'].tolist(), normalize_embeddings=True)

In [51]:
embeddings_test_dar_normed = pretrained_model.encode(dtest_darija['text'].tolist(), normalize_embeddings=True)
embeddings_test_ar_normed = pretrained_model.encode(dtest_arabe['text'].tolist(), normalize_embeddings=True)

In [50]:
embeddings_normed.shape

(5780, 512)

In [6]:

from laserembeddings import Laser
laser = Laser()
embeddings = laser.embed_sentences(
    ['I love pasta.',
     "J'adore les pâtes.",
     'Ich liebe Pasta.', 'بسم الله'],
    lang=['en', 'fr', 'de', 'ar'])

In [7]:
ground_truths = df['label'].tolist()
embeddings = laser.embed_sentences(df['text'].tolist(), lang=df['language'].tolist())

In [21]:
embeddings_test_dar = laser.embed_sentences(dtest_darija['text'].tolist(), lang=dtest_darija['language'].tolist())
embeddings_test_ar = laser.embed_sentences(dtest_arabe['text'].tolist(), lang=dtest_arabe['language'].tolist())

In [8]:
embeddings.shape

(5780, 1024)

In [22]:
import numpy.linalg as LA
nrm = LA.norm(embeddings, axis=1)
nrm_test_dar = LA.norm(embeddings_test_dar, axis=1)
nrm_test_ar = LA.norm(embeddings_test_ar, axis=1)

In [18]:
nrm.shape, nrm_reshaped.shape

((5780,), (5780, 1))

In [19]:
import numpy as np
nrm_reshaped = nrm[:, np.newaxis]
# Perform the division
embeddings_normed = embeddings / nrm_reshaped
#embeddings_normed = embeddings/nrm
embeddings_normed.shape

(5780, 1024)

In [23]:
import numpy as np
nrm_test_dar_reshaped = nrm_test_dar[:, np.newaxis]
embeddings_test_dar_normed = embeddings_test_dar / nrm_test_dar_reshaped

In [24]:
import numpy as np
nrm_test_ar_reshaped = nrm_test_ar[:, np.newaxis]
embeddings_test_ar_normed = embeddings_test_ar / nrm_test_ar_reshaped

In [30]:
len(df), len(ground_truths)

(5780, 5780)

In [52]:
import faiss

index = faiss.IndexFlatL2(embeddings_normed.shape[1])
index.add(embeddings_normed)

query = embeddings_test_ar_normed
#y = y_test_darija
k = 1 # number of nearest neighbors to return
D, I = index.search(query, k)

In [53]:
predicted_intents_for_ar = [ground_truths[x[0]] for x in I.tolist()]

In [54]:
import faiss

index = faiss.IndexFlatL2(embeddings_normed.shape[1])
index.add(embeddings_normed)

query = embeddings_test_dar_normed
#y = y_test_darija
k = 1 # number of nearest neighbors to return
D, I = index.search(query, k)

In [55]:
predicted_intents_for_dar = [ground_truths[x[0]] for x in I.tolist()]

In [56]:
ground_truths_test_ar = dtest_arabe['label'].tolist()
ground_truths_test_dar = dtest_darija['label'].tolist()

In [57]:
acc_msa, prec_msa, rec_msa, f1_msa = compute_metrics(ground_truths_test_ar, predicted_intents_for_ar)
acc_dar, prec_dar, rec_dar, f1_dar = compute_metrics(ground_truths_test_dar, predicted_intents_for_dar)

In [58]:
print('\n'.join([str(x) for x in [acc_msa, prec_msa, rec_msa, f1_msa, acc_dar, prec_dar, rec_dar, f1_dar]]))

0.850828729281768
0.8524941477582731
0.8388314871045136
0.8417692036873037
0.8425414364640884
0.8397650688091866
0.829782455604824
0.8298281170134305


In [39]:
acc_dar, prec_dar, rec_dar, f1_dar

(0.8535911602209945, 0.847813529063529, 0.8408276969132235, 0.8384561228426461)